In [1]:
import pandas as pd
import re
import datetime
import numpy as np
import pickle

In [2]:
!python -V

Python 3.6.4 :: Anaconda custom (64-bit)


In [3]:
print("Pandas version:",pd.__version__)
print("Numpy version:",np.__version__)

Pandas version: 0.22.0
Numpy version: 1.13.3


# Getting and Cleaning the Data
Getting the data and clearing extra spaces from the column names

In [4]:
# urls = ['http://web.mta.info/developers/data/nyct/turnstile/turnstile_170304.txt',
#         'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170311.txt',
#         'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170318.txt',
#         'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170325.txt',
#         'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170401.txt',
#         'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170408.txt',
#         'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170415.txt',
#         'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170422.txt',
#         'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170429.txt',
#         'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170506.txt',
#         'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170513.txt',
#         'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170520.txt',
#         'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170527.txt]

url = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170304.txt'

df = pd.read_csv(url, delimiter = ',')
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/25/2017,03:00:00,REGULAR,6064627,2055986
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/25/2017,07:00:00,REGULAR,6064645,2055999
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/25/2017,11:00:00,REGULAR,6064712,2056102
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/25/2017,15:00:00,REGULAR,6064903,2056172
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/25/2017,19:00:00,REGULAR,6065267,2056245


In [5]:
df.to_pickle('Untouched_Data.pkl', compression='gzip')
#df = pd.read_pickle('Untouched_Data.pkl', compression='gzip')

In [6]:
# Making the column names "pretty"
Cols_Pretty_Name = {}
for c in df.columns:
    Cols_Pretty_Name[c] = c.strip()
df = df.rename(columns=Cols_Pretty_Name)

## Turnstile Assumptions
We are assuming each turnstile in each station is represented by the variables unit and scp, where scp stands for subunit/channel/location.  The variable unit represents the remote ID of a station and as there are more units than stations, we are assuming some stations have multiple units. The variable SCP represents subunit/channel/location.  For the stations with multiple units we have found that different units have the same SCP value; therefore, we are assuming each turnstile is represented by a combination of the unit and the SCP value.  Hence the unique identifier for each turnstile in each station will be represented by the unit and SCP value.

In [7]:
df['DATE_AND_TIME'] = df[['DATE', 'TIME']].apply(lambda x: ' '.join(x), axis=1)
df['UNIT_AND_SCP'] = df[['UNIT', 'SCP']].apply(lambda x: ' '.join(x), axis=1)

# Resizing to only columns that are needed
df = df[['UNIT_AND_SCP','STATION','DATE_AND_TIME','ENTRIES','EXITS']]

# Converting the date and time column into a datetime object
df['DATE_AND_TIME']=pd.to_datetime(df['DATE_AND_TIME'], format='%m/%d/%Y %H:%M:%S')
df.head()

,UNIT AND SCP,STATION,DATE AND TIME,ENTRIES,EXITS
0,R051 02-00-00,59 ST,2017-02-25 03:00:00,6064627,2055986
1,R051 02-00-00,59 ST,2017-02-25 07:00:00,6064645,2055999
2,R051 02-00-00,59 ST,2017-02-25 11:00:00,6064712,2056102
3,R051 02-00-00,59 ST,2017-02-25 15:00:00,6064903,2056172
4,R051 02-00-00,59 ST,2017-02-25 19:00:00,6065267,2056245


In [8]:
df.to_pickle('Simplified_Data.pkl', compression='gzip')
#df = pd.read_pickle('Simplified_Data.pkl', compression='gzip')

# Weighted Average Function
For each turnstile the cumulative number of riders is put into the following 4 hour time bins for each day: 0, 4, 8, 12, 16, and 20.  For turnstiles that do not have a value that falls on one of those hours, then a weighted average will be used to approximate the missing data.  This average will use the data for the hours that fall just below, $x_r$, and just above, $x_l$, the desired time.  The weights will be taken as one over the distance from the desired number, $x$, and normalized; therefore the weighted average, $\bar c$, will be 
$$\bar{c} = \frac{1}{N}\left(\frac{1}{x-x_r}c_r+\frac{1}{x_l-x}c_l\right)$$
where $c_{r}$ and $c_{l}$ are the cumulative values for the lower and upper samples, respectivley, and the $N$ is the normalizing constant
$$N = \frac{1}{x-x_r}+\frac{1}{x_l-x}.$$
Note: Simplifying the weights divided by the normalizing constant gives
$$\bar{c} = \frac{x_l-x}{x_l-x_r}c_r+\frac{x-x_r}{x_l-x_r}c_l$$

In [9]:
def weighted_average(df):
    df = df.to_frame(name='Number')
    df['Date'] = df.index.values
    
    ''' Resample into 4 hour bins with the interval closed on the left side and the label is the left number; 
     hence, the bin 0 to 4 is labeled 0 and is [0,4)
     The first valid data value in the interval is used to represent the interval. In the interval [0,4), the
     the time representing the first valid data value will be larger than the time representing the interval
     _L denotes the object is for the number to the right (larger time) '''
    df_L = df.resample('4H',label='left',closed='left').first()
    df_L = df_L.rename(columns={'Date':'Right_Date','Number':'Right_Number'})
    
    ''' Resample into 4 hour bins with the interval closed on the right side and the label is the right number; 
     hence, the bin 0 to 4 is labeled 4 and is (0,4]
     The last valid data value in the interval is used to represent the interval. In the interval (0,4], the
     the time representing the last valid data value will be smaller than the time representing the interval
     _R denotes the object is for the number to the left (smaller time) '''
    df_R=df.resample('4H',label='right',closed='right').last()
    df_R = df_R.rename(columns={'Date':'Left_Date','Number':'Left_Number'})
    
    df = pd.concat([df_R,df_L],axis=1)
    
    # The differences mentioned in the equation for c_bar
    df['Difference'] = (df['Right_Date']-df['Left_Date'])/ np.timedelta64(1, 'h')    
    df['Right_Differenece'] = (df['Right_Date']-df.index.values)/ np.timedelta64(1, 'h')    
    df['Left_Difference'] = (df.index.values-df['Left_Date'])/ np.timedelta64(1, 'h')    

    def get_weights(x,y):
        if y != 0:
            return x/y
        else:
            return 1/2
    
    # Add weights needed for the weighted average to table
    df['Left_Weights'] = df.apply(lambda x: get_weights(x['Right_Differenece'], x['Difference']), axis=1)
    df['Right_Weights'] = df.apply(lambda x: get_weights(x['Left_Difference'], x['Difference']), axis=1)
    
    # Calculate the weighted average
    df['Weighted_Average'] = df['Left_Number']*df['Left_Weights']+df['Right_Number']*df['Right_Weights']
    df = df['Weighted_Average']
    
    return df

# Applying the weighted average function
Standardizing the ridership data to be every four hours, then finding the point mass values using the cumulative values. If there are time slots with missing information for a subgroup of terminals in one station, then the average of the ridership of the remaining terminals at that station is used. This assumes a uniform distribution among the turnstiles and entrances/exits are not closed off.
The data for each turnstile is stored in a dictionary using the station as the key.

In [10]:
Station_Names = df['STATION'].unique()
Station_Ridership_Data = {}
for station in Station_Names:
    df_S = df.loc[df['STATION']==station]
    for k in ['ENTRIES','EXITS']:
        df_S1 = df_S[['DATE_AND_TIME','UNIT_AND_SCP',k]]
        df_S1 = df_S1.drop_duplicates(subset = ['DATE_AND_TIME','UNIT_AND_SCP'],keep = 'last')
        df_S1 = df_S1.pivot(index = 'DATE_AND_TIME',columns = 'UNIT_AND_SCP',values = k)

        df_S1 = df_S1.apply(weighted_average,axis=0)

        # PMF of number of people through the turnstiles at each time point.
        df_S1 = abs(df_S1-df_S1.shift(1))
        df_S1 = df_S1[1:]

        # Fills missing data with the average along the row (same time different turnstiles)
        df_S1 = df_S1.apply(lambda row: row.fillna(row.mean()), axis=1)

        Station_Ridership_Data[station+' '+k] = df_S1


In [19]:
with open('Station_Ridership_Data.pkl','wb') as pickle_out:
    pickle.dump(Station_Ridership_Data, pickle_out)

# with open('Station_Ridership_Data.pkl','rb') as pickle_in:
#     Station_Ridership_Data = pickle.load(pickle_in)

While having individual data for each turnstile could be interesting, for our purposes it is better to obtain a total output for each station.  While summing the data at first seems like the obvious choice, there are outliers to contend with. The standard method would be to use the $1.5\times IQR$ to define our 'normal' observations, replace the 'non-normal' observations with the mean or median of the remaining values, and then sum the total number of entries and exits. However, we are assuming each turnstile has an approximately equal output, so a simple approach is to take the median (which is unaffected by outliers) and multiply it by the number of turnstiles in the station.

In [20]:
# SRD represents Station_Ridership_Data
def Total_Ridership_Per_Station(SRD):
    Tot_Riders_Per_Station = pd.DataFrame()
    for station in Station_Names:
        number_of_turnstiles = len(station.columns)
        df_Ent = SRD[station+' '+'ENTRIES'].median(axis=1)*number_of_turnstiles
        df_Ext = SRD[station+' '+'EXITS'].median(axis=1)*number_of_turnstiles
        df_Tot = df_Ent+df_Ext
        Tot_Riders_Per_Station = pd.concat([Tot_Riders_Per_Station,df_Tot.to_frame(station)],axis=1)
            
    return Tot_Riders_Per_Station

T = Total_Ridership_Per_Station(Station_Ridership_Data)
T.head()

,59 ST,5 AV/59 ST,57 ST-7 AV,49 ST,TIMES SQ-42 ST,34 ST-HERALD SQ,28 ST,23 ST,14 ST-UNION SQ,8 ST-NYU,...,BEVERLY RD,NEWKIRK AV,FLATBUSH AV-B.C,MORRIS PARK,BAYCHESTER AV,EASTCHSTER/DYRE,ST. GEORGE,TOMPKINSVILLE,RIT-MANHATTAN,RIT-ROOSEVELT
DATE AND TIME,,,,,,,,,,,,,,,,,,,,,
2017-02-25 04:00:00,0.00,0.00,0.00,0.00,0.00,0.00,1679.535714,2704.65625,8411.351351,0.00,...,0.00,388.0,775.0,98.0,86.0,218.0,0.00,0.00,189.0,57.0
2017-02-25 08:00:00,4757.00,1095.00,2784.75,1963.75,6756.75,6043.50,3068.500000,6050.00000,4979.027027,1081.25,...,377.00,814.0,1595.0,159.0,272.0,454.0,14.75,528.25,69.0,90.0
2017-02-25 12:00:00,13340.75,3185.25,8184.00,5875.50,19583.75,18747.75,8854.500000,18784.00000,21702.108108,3569.50,...,721.25,1499.0,3298.0,389.0,441.0,803.0,16.25,679.50,560.0,713.0
2017-02-25 16:00:00,18878.00,5508.00,11995.00,9602.00,32504.25,31896.75,11999.000000,28260.00000,41750.000000,7270.75,...,743.00,1598.0,3985.0,484.0,467.0,962.0,18.75,797.00,1231.0,1348.0
2017-02-25 20:00:00,18443.00,5025.25,12695.50,11528.00,38696.50,34698.25,11627.000000,25425.50000,46052.000000,8988.00,...,696.25,1629.0,3975.0,485.0,375.0,885.0,5.75,810.50,1109.0,928.0


# Unpacking the data
Unpacking the data to to have the columns: Station, Total Traffic, Date and Time.  This is for use in another program.

In [21]:
T['DATE_AND_TIME'] = T.index
T = pd.melt(T,id_vars=['DATE_AND_TIME'],var_name='STATION',value_name='TOTAL_TRAFFIC')
T['DATE_AND_TIME'] = T['DATE_AND_TIME'].dt.strftime('%m/%d/%Y %H:%M:%S')
T['DATE'],T['TIME'] = T['DATE_AND_TIME'].str.split(' ', 1).str
T = T.drop('DATE_AND_TIME',axis=1)

T.to_pickle('Total_Ridership_Per_Station.pkl', compression='gzip')

T.head()

,DATE AND TIME,STATION,TOTAL TRAFFIC,DATE,TIME
0,02/25/2017 04:00:00,59 ST,0.00,02/25/2017,04:00:00
1,02/25/2017 08:00:00,59 ST,4757.00,02/25/2017,08:00:00
2,02/25/2017 12:00:00,59 ST,13340.75,02/25/2017,12:00:00
3,02/25/2017 16:00:00,59 ST,18878.00,02/25/2017,16:00:00
4,02/25/2017 20:00:00,59 ST,18443.00,02/25/2017,20:00:00
